In [29]:
from datetime import datetime, timedelta

import awswrangler as wr
import boto3
import pandas as pd
import numpy as np
from faker import Faker

fake = Faker()

In [78]:
np.random.randint(low = 2000, high = 2022, size = 1)[0]

2009

In [106]:
import random

def rand_date():
    ex_date = datetime(np.random.randint(low = 2000, high = 2022, size = 1)[0],
                np.random.randint(low = 1, high = 12, size = 1)[0],
                np.random.randint(low = 1, high = 28, size = 1)[0])
    return ex_date

rand_date()

datetime.datetime(2004, 4, 6, 0, 0)

In [8]:
df = pd.DataFrame()
df['data1'] = np.random.randint(low = 1, high = 1000, size = 1000000)
df['data2'] = np.random.randint(low = 1001, high = 2000, size = 1000000)
df['data3'] = np.random.randint(low = 2001, high = 3000, size = 1000000)
df['data4'] = np.random.randint(low = -3001, high = 4000, size = 1000000)
df['data5'] = np.random.randint(low = -4001, high = 5000, size = 1000000)
df['date_ts'] = df.apply(lambda x: rand_date(), axis = 1)
df['name'] = df.apply(lambda x: fake.name(), axis = 1)

In [108]:
file_name = 'jacobsbucket97-dev/parquet_test/test_textdate.parquet'
file_name_snappy = 'jacobsbucket97-dev/parquet_test/test_textdate_snappy.snappy'
file_name_gzip = 'jacobsbucket97-dev/parquet_test/test_textdate_gzip.gzip'

s3 = boto3.resource('s3')

In [116]:
wr.s3.to_parquet(df=df, path = f's3://{file_name}')

{'paths': ['s3://jacobsbucket97-dev/parquet_test/test_textdate.parquet'],
 'partitions_values': {}}

In [109]:
## awswrangler uses SNAPPY by default on this function
wr.s3.to_parquet(df=df, path = f's3://{file_name}', compression = None)
wr.s3.to_parquet(df=df, path = f's3://{file_name_snappy}', compression = 'snappy')
wr.s3.to_parquet(df=df, path = f's3://{file_name_gzip}', compression = 'gzip')

{'paths': ['s3://jacobsbucket97-dev/parquet_test/test_textdate_gzip.gzip'],
 'partitions_values': {}}

# NOTES
I stored a dataframe of 1,000,000 records with 5, 6, and 7 different columns to test S3 storage scaling.

So integer data for the 5 data columns ALL stored at the exact same storage size at roughly ~6.9 MB, regardless of the compression used.

Adding 1 additional text column with generic name data dramatically increased the storage size.

Adding 1 additional static timestamp column DIDNT CHANGE THE DATA SIZE.  Because it's compressed and the 1,000,000 are all the same value.

Adding 1 additional dynamic timestamp column DID change the size but not as much as the text.

## GZIP
    no_text:         6.9 MB
    text:           12.2 MB
    text+samedatee: 12.2 MB
    text+randdatee: 13.8 MB

## Snappy
    no_text:         6.9 MB
    text:           15.5 MB
    text+samedatee: 15.5 MB
    text+randdatee: 17.1 MB

## No Compression
    no_text:         7.0 MB
    text:           23.2 MB
    text+samedatee: 23.2 MB
    text+randdatee: 24.9 MB

# Conclusions
Integer data is pretty efficient.  Text is expensive.  Dates are super efficient.  You get incredible compression savings for storing the exact same values for certain columns.

In [114]:
# there's variability in load times bc of network shit, think they're all the same essentially
df_s3 = wr.s3.read_parquet(f's3://{file_name}') # 3.6 seconds
df_s3_snappy = wr.s3.read_parquet(f's3://{file_name_snappy}') # 2.8
df_s3_gz = wr.s3.read_parquet(f's3://{file_name_gzip}') # 3.9 seconds 